# Beyond explicit matrix representations

# Overview

* Abstract linear operators in Python
* Some example with PyLops

# Abstract linear operators in Python

* So far, we have explicitly represented linear operators as a matrix
* For many algorithms we only need to be able to apply the operator and its adjoint to given input vectors

# An example with PyLops

## A linear operator for convolution

See https://pylops.readthedocs.io/en/stable/tutorials/deblurring.html

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pylops

ModuleNotFoundError: No module named 'pylops'

## Regularised least-squares

## Total Variation

# Exercise

Think how your own inverse proble could be cast in this framework